In [19]:
IFD_TOKN = "PPrBwtUvQCWRGBzfQa3TYgcuuFA9EYHJ7r3OKg4VhU9D1zM79oV07rBKxa_Hd-74TOzHKCTZK4S9SAypvo_VyA=="
IFD_ORGN = "artica"
IFD_HOST = "eu-central-1-1.aws.cloud2.influxdata.com"
IFD_ADDR = f"https://{IFD_HOST}"

grafanaToken = "HgPeKvrZrR75TnyDNPbkaMg2wMKRdX7wXPGttlI7-QUmei5dvfxXZD-tnHVpbSxXQNGUF2_U-o4z498qWMQLvw=="

import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS

client = influxdb_client.InfluxDBClient(
   url=IFD_ADDR,
   token=IFD_TOKN,
   org=IFD_ORGN
)

write_api = client.write_api(write_options=SYNCHRONOUS)

In [ ]:
# 2023-03-18 17:37:40.807603965
# 2023-03-18 17:37:46.291760032

In [24]:
client.health()

{'checks': None,
 'commit': None,
 'message': 'Invalid value for `name`, must not be `None`',
 'name': 'influxdb',
 'status': 'fail',
 'version': None}